# 🚀 Real-time Bitcoin Price Prediction Dashboard

**TDA + Machine Learning** 기반 실시간 가격 예측 시스템

---

In [ ]:
# Import libraries
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, HTML, clear_output
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded!")

## 📊 Check Model Status

In [ ]:
# Load training metrics
try:
    with open('training_metrics.json', 'r') as f:
        metrics = json.load(f)
    
    print("="*60)
    print("🤖 MODEL TRAINING METRICS")
    print("="*60)
    print(f"Trained at: {metrics['trained_at']}")
    print(f"\nPrediction Horizon: {metrics['forecast_horizon_minutes']} minutes")
    print(f"Window Size: {metrics['window_size_minutes']} minutes")
    print(f"\nPerformance:")
    print(f"  Train MAE:  ${metrics['train_mae']:.2f}")
    print(f"  Test MAE:   ${metrics['test_mae']:.2f}")
    print(f"  Train RMSE: ${metrics['train_rmse']:.2f}")
    print(f"  Test RMSE:  ${metrics['test_rmse']:.2f}")
    print(f"  Train R²:   {metrics['train_r2']:.4f}")
    print(f"  Test R²:    {metrics['test_r2']:.4f}")
    print("="*60)
    
    # Display as HTML
    html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
        <h2 style="margin: 0;">🤖 Model Performance</h2>
        <p style="margin: 10px 0 0 0; font-size: 18px;">Test MAE: ${metrics['test_mae']:.2f} | Test R²: {metrics['test_r2']:.4f}</p>
        <p style="margin: 5px 0 0 0; opacity: 0.9;">Predicting {metrics['forecast_horizon_minutes']} minutes ahead</p>
    </div>
    """
    display(HTML(html))
    
except FileNotFoundError:
    print("⚠️ Model not trained yet. Run realtime_predictor.py first!")

## 🔴 Real-time Prediction Status

In [ ]:
# Load live prediction status
def load_live_status():
    try:
        with open('live_prediction_status.json', 'r') as f:
            status = json.load(f)
        return status
    except FileNotFoundError:
        return None

status = load_live_status()

if status:
    pred = status['prediction']
    prices = status['current_prices']
    
    # Display banner
    change_pct = pred['predicted_change_pct']
    if change_pct > 0:
        color = '#4caf50'
        symbol = '📈 UP'
    elif change_pct < 0:
        color = '#f44336'
        symbol = '📉 DOWN'
    else:
        color = '#9e9e9e'
        symbol = '➡️ FLAT'
    
    html = f"""
    <div style="background-color: {color}; padding: 25px; border-radius: 10px;
                color: white; margin: 20px 0; border-left: 8px solid #333;">
        <h2 style="margin: 0; font-size: 32px;">{symbol}</h2>
        <p style="margin: 15px 0 0 0; font-size: 24px; font-weight: bold;">
            ${pred['current_price']:,.2f} → ${pred['predicted_price']:,.2f}
        </p>
        <p style="margin: 10px 0 0 0; font-size: 20px;">
            {change_pct:+.2f}% ({pred['predicted_change']:+,.2f} USDT)
        </p>
        <p style="margin: 15px 0 0 0; font-size: 14px; opacity: 0.9;">
            Prediction for {pred['forecast_horizon_minutes']} minutes ahead
        </p>
        <p style="margin: 5px 0 0 0; font-size: 14px; opacity: 0.8;">
            Last update: {status['timestamp']}
        </p>
    </div>
    """
    display(HTML(html))
    
    # Metrics grid
    metrics_html = f"""
    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin: 20px 0;">
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">💰 Current Price</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">${pred['current_price']:,.2f}</div>
            <div style="font-size: 12px; margin-top: 5px;">Futures</div>
        </div>
        <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">🎯 Predicted Price</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">${pred['predicted_price']:,.2f}</div>
            <div style="font-size: 12px; margin-top: 5px;">{pred['forecast_horizon_minutes']} min ahead</div>
        </div>
        <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
                    padding: 20px; border-radius: 10px; color: white; text-align: center;">
            <div style="font-size: 14px; opacity: 0.9;">📊 Expected Change</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">{change_pct:+.2f}%</div>
            <div style="font-size: 12px; margin-top: 5px;">${pred['predicted_change']:+,.2f} USDT</div>
        </div>
    </div>
    """
    display(HTML(metrics_html))
    
    # TDA Metrics
    tda_html = f"""
    <div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; margin: 20px 0;">
        <h3 style="margin: 0 0 15px 0;">📈 TDA Market Analysis</h3>
        <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px;">
            <div>
                <strong>L¹ Norm:</strong> {pred['tda_l1_norm']:.4f}
            </div>
            <div>
                <strong>L² Norm:</strong> {pred['tda_l2_norm']:.4f}
            </div>
            <div>
                <strong>Wasserstein:</strong> {pred['tda_wasserstein']:.4f}
            </div>
        </div>
    </div>
    """
    display(HTML(tda_html))
    
else:
    print("⚠️ No live prediction data available.")
    print("Start the live monitor with: python live_data_fetcher.py")

## 📊 Price Visualization

In [ ]:
if status:
    # Create gauge for prediction confidence
    fig = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=abs(pred['predicted_change_pct']),
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Predicted Change Magnitude", 'font': {'size': 24}},
        delta={'reference': 0, 'suffix': '%'},
        gauge={
            'axis': {'range': [None, 2], 'tickwidth': 1},
            'bar': {'color': color},
            'steps': [
                {'range': [0, 0.5], 'color': '#e8f5e9'},
                {'range': [0.5, 1.0], 'color': '#fff3e0'},
                {'range': [1.0, 2.0], 'color': '#ffebee'}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 1.0
            }
        }
    ))
    
    fig.update_layout(height=400)
    fig.show()

## 🔄 Auto-refresh Dashboard

Run this cell to continuously update the dashboard (Ctrl+C to stop)

In [ ]:
# Auto-refresh loop
try:
    refresh_interval = 60  # seconds
    
    while True:
        clear_output(wait=True)
        
        # Load latest status
        status = load_live_status()
        
        if status:
            pred = status['prediction']
            
            change_pct = pred['predicted_change_pct']
            if change_pct > 0:
                symbol = '📈 UP'
            elif change_pct < 0:
                symbol = '📉 DOWN'
            else:
                symbol = '➡️ FLAT'
            
            print("="*80)
            print(f"🚀 REAL-TIME BITCOIN PREDICTION - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print("="*80)
            print(f"\n{symbol}")
            print(f"\nCurrent Price:   ${pred['current_price']:,.2f}")
            print(f"Predicted Price: ${pred['predicted_price']:,.2f}")
            print(f"Expected Change: {change_pct:+.2f}% (${pred['predicted_change']:+,.2f})")
            print(f"\nPrediction Time: {pred['prediction_time']} ({pred['forecast_horizon_minutes']} min ahead)")
            print(f"\nTDA Analysis:")
            print(f"  L¹ Norm: {pred['tda_l1_norm']:.4f}")
            print(f"  L² Norm: {pred['tda_l2_norm']:.4f}")
            print(f"  WD:      {pred['tda_wasserstein']:.4f}")
            print(f"\nLast Update: {status['timestamp']}")
            print("="*80)
        else:
            print("⚠️ Waiting for live prediction data...")
        
        print(f"\n⏱️  Next refresh in {refresh_interval} seconds...")
        print("(Ctrl+C to stop)")
        
        time.sleep(refresh_interval)
        
except KeyboardInterrupt:
    print("\n✋ Auto-refresh stopped")

## 📝 Usage Instructions

### 1. Train Model (First Time)
```bash
cd /notebooks/tda-extreme-events
python realtime_predictor.py
```

### 2. Start Live Monitoring
```bash
python live_data_fetcher.py --interval 300
```

### 3. View Dashboard
- Run cells above to see current prediction
- Run auto-refresh cell for continuous updates

---

**System Components:**
- `realtime_predictor.py`: Train TDA + ML model
- `live_data_fetcher.py`: Fetch real-time Binance data & predict
- `live_dashboard.ipynb`: This dashboard (visualization)

**Data Sources:**
- Binance Futures API (BTCUSDT Perpetual)
- Binance Spot API (BTCUSDT)

**Features:**
- TDA features (L¹, L², Wasserstein distance)
- Price & volume from Futures + Spot
- Technical indicators (MA, volatility, etc.)

**Model:**
- XGBoost Regressor
- Predicts 60 minutes ahead
- Trained on 100,000+ 5-minute candles